# Linear Quadratic Regulator

The previous section showed that the eigenvalues of the stable system regarding the angle and angular velocity of the pole have characteristics of continuous systems rather than discrete systems.

We solve the linear quadratic regulator problem by solving the continuous Ricatti equation.

In [1]:
import gym
import tensorflow as tf
import numpy as np
import pickle
import control

In [2]:
model = tf.keras.models.load_model(
    './cartpole_system_model', custom_objects=None, compile=True, options=None
)
np_weights = model.get_weights()

A = np_weights[0]
B = np_weights[1].T
print("A Matrix")
print(A)
print("B Matrix")
print(B)

A Matrix
[[ 1.0000035e+00 -1.2942681e-05 -2.3801964e-05  1.6305943e-03]
 [ 2.0008639e-02  9.9992085e-01 -2.6311722e-05 -9.5092575e-04]
 [-8.2056704e-07 -1.3424688e-02  1.0000260e+00  3.1254122e-01]
 [-4.4493249e-06 -5.0352475e-05  2.0024499e-02  9.9951327e-01]]
B Matrix
[[ 6.6651064e-06]
 [ 1.9508155e-01]
 [-1.1186228e-05]
 [-2.9142728e-01]]


2022-04-10 16:55:27.254677: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Q = 

# Continuous Algebraic Ricatti Equation solution
S, Lambda_S, K_gain = control.care(A, B, Q, R=None)